# Этап 4: Реальное квантование (Weight-only INT8 Storage)

Здесь мы уходим от симуляции и физически меняем тип данных в памяти. Мы будем хранить веса как `int8`, а для вычислений — превращать их обратно в `float16/32` на лету.

In [4]:
import torch
import os
from src.model import GPTLanguageModel, device, encode, decode

# 1. Загружаем оригинал
model = GPTLanguageModel().to(device)
model.load_state_dict(torch.load('model_ckpt.pt', map_location=device))
model.eval()

# Функции для реального квантования в словарь
def compress_to_int8(state_dict):
    compressed_dict = {}
    scales = {}
    
    for name, param in state_dict.items():
        if 'weight' in name and param.dim() > 1:
            # Находим масштаб
            x_max = param.abs().max().item()
            scale = x_max / 127.0
            
            # Физически кастуем в int8
            q_param = torch.round(param / scale).to(torch.int8)
            
            compressed_dict[name] = q_param
            scales[name + "_scale"] = torch.tensor(scale)
            print(f"📦 Сжат слой {name}: {param.dtype} -> {q_param.dtype}")
        else:
            # Остальное (LayerNorm, Bias) оставляем как есть
            compressed_dict[name] = param
            
    return compressed_dict, scales

compressed_weights, scales = compress_to_int8(model.state_dict())

📦 Сжат слой token_embedding_table.weight: torch.float32 -> torch.int8
📦 Сжат слой position_embedding_table.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.0.key.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.0.query.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.0.value.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.1.key.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.1.query.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.1.value.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.2.key.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.2.query.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.2.value.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.3.key.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.3.query.weight: torch.float32 -> torch.int8
📦 Сжат слой blocks.0.sa.heads.3.value.weight: torch

### 2. Сравнение размера файлов на диске
Это самый честный способ проверить сжатие.

In [5]:
torch.save(model.state_dict(), 'model_fp32.pt')
torch.save({'weights': compressed_weights, 'scales': scales}, 'model_int8_real.pt')

size_fp32 = os.path.getsize('model_fp32.pt') / 1024**2
size_int8 = os.path.getsize('model_int8_real.pt') / 1024**2

print(f"Размер файла FP32: {size_fp32:.2f} MB")
print(f"Размер файла INT8: {size_int8:.2f} MB")
print(f"Реальная экономия места: {size_fp32/size_int8:.1f}x")

Размер файла FP32: 50.23 MB
Размер файла INT8: 19.46 MB
Реальная экономия места: 2.6x


### 3. Как запустить такую модель? (Dequantize-on-the-fly)
Мы не можем просто сделать `model.load_state_dict()`, так как типы не совпадут. Нам нужно деквантовать веса перед загрузкой.

In [6]:
def decompress_int8(compressed_data):
    weights = compressed_data['weights']
    scales = compressed_data['scales']
    new_state_dict = {}
    
    for name, param in weights.items():
        if name + "_scale" in scales:
            # Восстанавливаем из int8 во float32
            scale = scales[name + "_scale"]
            new_state_dict[name] = param.to(torch.float32) * scale
        else:
            new_state_dict[name] = param
            
    return new_state_dict

# Эмуляция загрузки сжатой модели
checkpoint = torch.load('model_int8_real.pt', map_location=device)
recovered_state = decompress_int8(checkpoint)

model_int8_recovered = GPTLanguageModel().to(device)
model_int8_recovered.load_state_dict(recovered_state)
model_int8_recovered.eval()

print("✅ Модель успешно восстановлена из INT8 и готова к работе!")

✅ Модель успешно восстановлена из INT8 и готова к работе!


### 4. Сравнение генерации (Baseline vs Recovered)
Убедимся, что после физического цикла сжатия-распаковки качество осталось таким же, как при Fake Quantization.

In [ ]:
prompt = "JULIET: "
context = torch.tensor(encode(prompt), dtype=torch.long, device=device).unsqueeze(0)

print("--- ORIGINAL FP32 OUTPUT ---")
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))

print("\n--- RECOVERED INT8 OUTPUT ---")
print(decode(model_int8_recovered.generate(context, max_new_tokens=500)[0].tolist()))

--- ORIGINAL FP32 OUTPUT ---
JULIET: too woman! O noble lords!
I should fit were 't person your wife's son,
Foolish'd, may could not quit on true for them.

JULIET:
I thought by him your moon, do not think, I'll find you alone.

JULIET:
For I'll not this is his acception, 'Go and sure his King Richard's hand;
But, King Edward. In God forgetful true!
Ah, pilgrimagion or or heart's design to his love,
You not put unbroke itself in our tent spurses!
As the day of my sons; we show the father wrong'd him!
Hold not army diade gate and wi

--- RECOVERED INT8 OUTPUT ---
JULIET: and bring of heavier's eere so.

WARWICK:
O son, ext the way well; so look the fight,
And now, distu
